# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding-3233000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo-3233000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-31102000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = True, True

# you sit West and hold:
hand = '85.9.KQJT873.QJT'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
auction = []
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, False)

bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])
auction = ["PAD_START"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, False)

bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])
auction = ["PAD_START","PAD_START"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, False)

bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])
auction = ["PAD_START","PAD_START","PAD_START"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, False)

bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])

# what's your bid?

[{'call': '3D', 'insta_score': 0.9623}, {'call': '2D', 'insta_score': 0.0216}, {'call': 'PASS', 'insta_score': 0.0103}, {'call': '3N', 'insta_score': 0.0037}, {'call': '4D', 'insta_score': 0.0013}]
[{'call': '3D', 'insta_score': 0.9647}, {'call': '2D', 'insta_score': 0.0136}, {'call': 'PASS', 'insta_score': 0.0091}, {'call': '3N', 'insta_score': 0.007}, {'call': '4D', 'insta_score': 0.0039}, {'call': '5D', 'insta_score': 0.0011}]
[{'call': '3D', 'insta_score': 0.9623}, {'call': '2D', 'insta_score': 0.0216}, {'call': 'PASS', 'insta_score': 0.0103}, {'call': '3N', 'insta_score': 0.0037}, {'call': '4D', 'insta_score': 0.0013}]
[{'call': '3D', 'insta_score': 0.9647}, {'call': '2D', 'insta_score': 0.0136}, {'call': 'PASS', 'insta_score': 0.0091}, {'call': '3N', 'insta_score': 0.007}, {'call': '4D', 'insta_score': 0.0039}, {'call': '5D', 'insta_score': 0.0011}]


In [4]:
auction = ["PAD_START", "2D", "PASS", "PASS", "X", "PASS", "PASS"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': 'PASS', 'insta_score': 1.0}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [5]:
bid.samples

['AKQx.AQJTx..AKxx Jx.Kx.A9xxxx.98x T9xxx.8xxxx..xxx 8x.9.KQJT8xx.QJT 0.07125',
 'AKQxx.AKxxx..A8x Jx.Q8x.A9xxxx.Kx T9xx.JTxx..9xxxx 8x.9.KQJT8xx.QJT 0.06242',
 'AKTxx.AKQ8x..A9x xx.Txx.A9xxxx.Kx QJ9x.Jxxx..8xxxx 8x.9.KQJT8xx.QJT 0.03983',
 'AKJx.AKJxx..K8xx Q9.xx.A9xxxx.Axx Txxxx.QT8xx..9xx 8x.9.KQJT8xx.QJT 0.03391',
 'AKJ.AKJxxx..AKxx Qxx.Tx.A9xxxx.xx T9xxx.Q8xx..98xx 8x.9.KQJT8xx.QJT 0.03087',
 'AKT9.AKQxx..K8xx Qxx.8xx.A9xxxx.x Jxxx.JTxx..A9xxx 8x.9.KQJT8xx.QJT 0.03009',
 'AKQxxx.AQTxx..K9 Jx.x.A9xxxx.xxxx T9x.KJ8xxx..A8xx 8x.9.KQJT8xx.QJT 0.02665',
 'AQTxx.AKQxx..Kxx Jx.JTx.A9xxxx.xx K9xx.8xxx..A98xx 8x.9.KQJT8xx.QJT 0.02089',
 'Kxxx.AKQxx..AKxx A9.Jxx.A9xxxx.8x QJTxx.T8xx..9xxx 8x.9.KQJT8xx.QJT 0.01778',
 'AK9.AKJxx..AK8xx QTx.8x.A9xxxx.xx Jxxxx.QTxxx..9xx 8x.9.KQJT8xx.QJT 0.01670']

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [6]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp = bot_bid.sample_hands(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"])

for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

8x.9.KQJT8xx.QJT 9x.AJxxx.9xx.xxx AKQxxx.KTxx..AK9 JTx.Q8x.Axx.8xxx
8x.9.KQJT8xx.QJT Jxx.KTxx.x.8xxxx KQTxxx.QJxx.A.AK A9.A8xx.9xxx.9xx
8x.9.KQJT8xx.QJT xxx.KQJx.9x.8xxx AKQJ9x.ATxx..A9x Tx.8xxx.Axxx.Kxx
8x.9.KQJT8xx.QJT 9xx.KJTxx.Ax.xxx AQJTxx.Axxx..AK8 Kx.Q8x.9xxx.9xxx
8x.9.KQJT8xx.QJT J.A8xx.A9xx.8xxx AKQTxx.QTxx.x.AK 9xxx.KJxx.x.9xxx
8x.9.KQJT8xx.QJT xx.A8xx.9xx.Kxxx AKQ9xx.QJxx.Ax.A JTx.KTxx.x.98xxx
8x.9.KQJT8xx.QJT x.KTxx.xx.K98xxx AKQJT9x.AQ8x.A.A xxx.Jxxx.9xx.xxx
8x.9.KQJT8xx.QJT J.KTxxx.Axx.8xxx AKQTxx.AQJ8..K9x 9xxx.xxx.9xx.Axx
8x.9.KQJT8xx.QJT Txx.Axx.Ax.9xxxx AKQ9xx.KQJxx..AK Jx.T8xx.9xxx.8xx
8x.9.KQJT8xx.QJT xx.KJTxx.xx.K8xx AKQJ9xx.AQxx.A.A Tx.8xx.9xx.9xxxx
